# Проект №2. Исследование графа социальных связей

## Подключение необходимых библиотек

In [2]:
import numpy as np
import pandas as pd
import networkx as nx
import scipy.stats

%matplotlib inline
import matplotlib.pyplot as plt

## Задачи исследования

Для исследования возьмите датасет социальных связей с сайта [NetworkRepository](http://networkrepository.com/soc.php). Выберите датасет размером не менее 3 Мб и впишите свой выбор (название вида `soc-*`
) напротив фамилии в [форме](https://docs.google.com/spreadsheets/d/1V_36WnSxoSYTZ5ILrwCqqTd8SM0vXAha6t93nDAhtBc/edit?usp=sharing). Все датасеты должны быть уникальны в пределах группы.

**Задание**

1. Изучите библиотеку `networkx` и процесс построения графов в ней ([документация к библиотеке](https://networkx.github.io/documentation/stable/tutorial.html)). Загрузите выбранный граф социальных связей в ноутбук и выведите количество рёбер и узлов.
2. Любым способом определите наиболее важный узел (человека) и наиболее важное ребро (связь между людьми) в сети.
3. Определите, является ли граф связным.
4. Определите степень графа.
6. Проверьте теорию шести рукопожатий. Определите, какое минимальное и какое среднее количество рукопожатий связывает двух людей в графе.


**Дополнительные задачи**

1. Визуализируйте граф или наиболее существенную часть графа с помощью `networkx`


_NB. Исследовательские задания подразумевают неоднозначную (!) интерпретацию поставленных задач и, следовательно, могут иметь более одного верного решения_

Сдавать все результаты отправкой готового ноутбука персональным сообщением в [Rocket.Chat](https://chat.kazaryan.ml). Защита результатов проходит лично.

## Исследование

### 1.Нам необходимо заагрузить граф 

In [2]:
!wget http://nrvis.com/download/data/soc/soc-google-plus.zip 
!unzip -o soc-google-plus.zip

--2019-01-07 23:04:02--  http://nrvis.com/download/data/soc/soc-google-plus.zip
Resolving nrvis.com... 173.236.156.25
Connecting to nrvis.com|173.236.156.25|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6544810 (6.2M) [application/zip]
Saving to: 'soc-google-plus.zip.7'

soc-google-plus.zip 100%[===================>]   6.24M  4.03MB/s    in 1.6s    

2019-01-07 23:04:04 (4.03 MB/s) - 'soc-google-plus.zip.7' saved [6544810/6544810]

Archive:  soc-google-plus.zip
  inflating: soc-google-plus.txt     
  inflating: readme.html             


### 2.Первоначальная информация о графе:

In [3]:
GPlus = nx.read_edgelist('soc-google-plus.txt', create_using=nx.Graph(), nodetype= int)
print (nx.info(GPlus) )

Name: 
Type: Graph
Number of nodes: 211187
Number of edges: 1143411
Average degree:  10.8284


### Отрисовка графа 
#### Совет: Не запускать, отрисовывает граф полностью 

In [ ]:
from matplotlib import pylab 
import networkx as nx 

def save_graph(graph,file_name): 
    
    plt.figure(num=None, figsize=(20, 20), dpi=80) 
    plt.axis('off') 
    fig = plt.figure(1) 
    pos = nx.spring_layout(graph) 
    nx.draw_networkx_nodes(graph,pos) 
    nx.draw_networkx_edges(graph,pos) 
    nx.draw_networkx_labels(graph,pos) 

    cut = 1.00 
    xmax = cut * max(xx for xx, yy in pos.values()) 
    ymax = cut * max(yy for xx, yy in pos.values()) 
    plt.xlim(0, xmax) 
    plt.ylim(0, ymax) 
 
    plt.savefig(file_name,bbox_inches="tight") 
    pylab.close() 
    del fig 

save_graph(GPlus,"GPlus.png") 

### 2.1 Самый важный узел 

### Мы будем находить важность узла по следующей формуле:
<img src = 'Formula.png' height=100 weigth=100 >

In [3]:
from operator import itemgetter

degs = nx.degree_centrality(GPlus)
print('По убыванию:')
sorted(degs.items(), key = itemgetter(1), reverse= True)[:5]

По убыванию:


[(136198, 0.008475940640004545),
 (5381, 0.00719271163808207),
 (116002, 0.007098008390707717),
 (145647, 0.00663396247857339),
 (66836, 0.00645402630856212)]

In [4]:
print('По возрастанию:')
sorted(degs.items(), key = itemgetter(1), reverse= False)[:5]

По возрастанию:


[(6, 4.735162368717623e-06),
 (7, 4.735162368717623e-06),
 (12, 4.735162368717623e-06),
 (37, 4.735162368717623e-06),
 (40, 4.735162368717623e-06)]

### 2.2 Наиболее  важное ребро 

In [ ]:
edge_importance = nx.edge_betweenness_centrality(GPlus)

In [ ]:
sorted(edge_importance.items(), key = itemgetter(1))[:5]

#### Данная функция возвращает список ребер, которые соединяет наибольшее количество узлов в графе. Пример такого ребра: 

<img src  ='978-1-4419-9863-7_5_Part_Fig1-874_HTML-2.gif' ></img>

### Связный у нас граф? 

In [5]:
nx.is_connected(GPlus)

False

### Наш граф не является связным 

### Отрисовка графа случайно сгенерированного графа из исходного :

In [4]:
import random

In [5]:
lines = open('soc-google-plus.txt').read().splitlines()

In [6]:
google = []
for i in range(0,50):
    google.append(str(random.randint(1,20000)))

In [7]:
random_google = []
for line in open('soc-google-plus.txt'):
    rec = line.strip()
    if rec.startswith(tuple(google)):
        random_google.append(line)

In [ ]:
# Считывание
GRandom = nx.read_edgelist(random_google, create_using=nx.Graph(), nodetype= int)
print nx.info(GRandom)
# Отрисовка
plt.figure(num=None, figsize=(20, 20), dpi=80) 
pos_g = nx.spring_layout(GRandom ,iterations = 100)

cons = nx.constraint(GRandom)

plt.figure(figsize=(20,20))
nsize = np.array ([1-v for v in cons.values()])

nsize = 10**(nsize+1)

nodes = nx.draw_networkx_nodes (GRandom , pos = pos_g , node_color=nsize, cmap=plt.cm.coolwarm,
                                node_size = nsize)
edges = nx.draw_networkx_edges (GRandom , pos = pos_g , 
                                alpha = .1)

plt.axis('off')

Name: 
Type: Graph
Number of nodes: 13891
Number of edges: 15726
Average degree:   2.2642


/anaconda3/lib/python2.7/site-packages/networkx/drawing/layout.py:499: RuntimeWarning: invalid value encountered in sqrt
  distance = np.sqrt((delta**2).sum(axis=0))


### Важно:
#### Так как граф был получен случайным образом из исходного, о результатах мы можем  делать выводы с некоторой вероятностью 

### Теория шести рукопожатий

In [ ]:
shortest = nx.all_pairs_shortest_path_length(GRandom)
dist_each = dict(nx.all_pairs_shortest_path_length(GRandom))
max_dist = [max(v.values()) for k,v in dist_each.items()]
plt.hist(max_dist)
plt.xlabel('Ugliest girls graphic');